In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install -U ipywidgets
!pip install ultralytics

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

from matplotlib import pyplot as plt
from PIL import Image

import torch
from torch import nn


from ultralytics import YOLO
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("slavkoprytula/aquarium-data-cots")

print("Path to dataset files:", path)

In [ ]:
model = YOLO('yolov8s.pt')

#1. Are you adding any augmentations to the dataset. If yes, then which ones and why. If not, then why not?
#In this training setup, I didn’t add custom augmentations because YOLOv8 already includes default augmentations like flips, scaling, and rotations during training, which help make the model more robust to real-world variations. Given the nature of the aquarium dataset, these standard augmentations should cover most scenarios. 
#Additional augmentations might be unnecessary at this stage unless further tuning is needed after evaluating initial results.

In [ ]:
results = model.train(data='/kaggle/input/aquarium-data-cots/aquarium_pretrain/data.yaml', epochs=100, imgsz=640)

#2. How many epochs are you using for this dataset training?
#I chose to train the model for 100 epochs. This number balances the need for the model to learn enough without overtraining. It’s often a good starting point for moderately sized datasets and allows time to see clear performance patterns before committing to higher epoch counts.

In [ ]:
#3. List any other hyperparameters you are specifying while training. Give reasoning for same.
# Besides setting epochs to 100, I kept imgsz=640 for this training, as this image size works well to balance accuracy with computational efficiency. Higher resolutions could improve accuracy but might slow down the training significantly, so I’m aiming to keep things efficient here.

In [ ]:
metrics = model.val()

#4. What output metrices are you analysing. Share concrete results from those
# I’m tracking Precision, Recall, mAP@50, mAP@50-95, and Losses (Box, Objectness, and Classification). These will show how well the model detects and classifies objects accurately.

In [ ]:
def detection_pipline(images):
    output = model(images)
    
    # Visualize the results
    for i, r in enumerate(output):
        # Plot results image
        im_bgr = r.plot()  # BGR-order numpy array
        im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

        plt.axis('off')
        plt.imshow(im_rgb)
        plt.show()

In [ ]:
test_imgs = [
    "/kaggle/input/aquarium-data-cots/aquarium_pretrain/test/images/IMG_2289_jpeg_jpg.rf.fe2a7a149e7b11f2313f5a7b30386e85.jpg",
    "/kaggle/input/aquarium-data-cots/aquarium_pretrain/test/images/IMG_2301_jpeg_jpg.rf.2c19ae5efbd1f8611b5578125f001695.jpg",
    "/kaggle/input/aquarium-data-cots/aquarium_pretrain/test/images/IMG_2387_jpeg_jpg.rf.09b38bacfab0922a3a6b66480f01b719.jpg"
]

In [ ]:
detection_pipline(test_imgs)